<a href="https://colab.research.google.com/github/NathanDietrich/Artificial-Intelligence-and-Machine-Learning-portfolio/blob/main/Experimenting_with_various_models_stock_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grabbing Stock Data

In [1]:
!pip install yfinance --upgrade --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.49
    Uninstalling yfinance-0.2.49:
      Successfully uninstalled yfinance-0.2.49


In [39]:
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [3]:
#define range of study
end = dt.datetime.now()
start = end - dt.timedelta(days = 4*360)
start, end

(datetime.datetime(2020, 12, 16, 21, 44, 3, 591802),
 datetime.datetime(2024, 11, 25, 21, 44, 3, 591802))

In [10]:
#Choose stock tickers to study
stockList = ['NVDA', 'MSFT', 'AAPL', 'TSLA', 'AI', 'SMCI', 'ADI', 'NOW', 'DDOG', 'CROX', 'TWLO', 'ETSY', 'DXCM', 'ZBRA', 'ANET']
stocks = [i for i in stockList]
stocks

['NVDA',
 'MSFT',
 'AAPL',
 'TSLA',
 'AI',
 'SMCI',
 'ADI',
 'NOW',
 'DDOG',
 'CROX',
 'TWLO',
 'ETSY',
 'DXCM',
 'ZBRA',
 'ANET']

In [11]:
df = yf.download(stocks, start, end)

[*********************100%***********************]  15 of 15 completed


In [12]:
df.tail()

Price        Adj Close                                                 \
Ticker            AAPL         ADI         AI        ANET        CROX   
Date                                                                    
2024-11-19  228.279999  210.429993  32.959999  377.700012   97.980003   
2024-11-20  229.000000  211.009995  32.360001  381.709991   98.589996   
2024-11-21  228.520004  213.960007  34.990002  404.970001  101.809998   
2024-11-22  229.869995  214.589996  37.419998  406.480011  106.080002   
2024-11-25  232.869995  223.580002  37.860001  399.720001  108.529999   

Price                                                                  ...  \
Ticker            DDOG       DXCM       ETSY        MSFT          NOW  ...   
Date                                                                   ...   
2024-11-19  133.410004  74.879997  50.410000  416.955414  1022.099976  ...   
2024-11-20  135.449997  75.239998  50.860001  414.659973  1022.979980  ...   
2024-11-21  143.860001  74.459999  51.990002  412.869995  1047.050049  ...   
2024-11-22  154.830002  72.830002  52.299999  417.000000  1060.599976  ...   
2024-11-25  156.630005  77.830002  55.680000  418.790009  1052.709961  ...   

Price         Volume                                                  \
Ticker          DDOG     DXCM     ETSY      MSFT      NOW       NVDA   
Date                                                                   
2024-11-19   6252000  3351900  3345100  18133500  1830200  227834900   
2024-11-20   6353300  3077200  2234000  19191700  1141900  309871700   
2024-11-21   8671800  3230900  2337700  20780200  1111300  400946600   
2024-11-22  13874200  4562600  2377600  24802100  1436900  235772200   
2024-11-25   7654744  6556778  6579015  25842761   998346  326414452   

Price                                             
Ticker           SMCI      TSLA     TWLO    ZBRA  
Date                                              
2024-11-19  235282900  88852500  2292100  324500  
2024-11-20  105366500  66340700  2763000  489400  
2024-11-21  118032600  58011700  4216400  368400  
2024-11-22  159205900  88896400  2502500  232000  
2024-11-25  166191735  93338715  5913672  384151  

[5 rows x 90 columns]

In [20]:
print(df.info)

<bound method DataFrame.info of Price        Adj Close                                                  \
Ticker            AAPL         ADI          AI        ANET        CROX   
Date                                                                     
2020-12-17  125.849930  134.495972  117.239998   71.342499   65.269997   
2020-12-18  123.855118  135.225555  137.589996   71.930000   64.379997   
2020-12-21  125.390350  133.382828  160.889999   71.815002   65.620003   
2020-12-22  128.959518  134.963684  177.470001   72.705002   64.760002   
2020-12-23  128.059906  133.972137  161.059998   71.622498   64.589996   
...                ...         ...         ...         ...         ...   
2024-11-19  228.279999  210.429993   32.959999  377.700012   97.980003   
2024-11-20  229.000000  211.009995   32.360001  381.709991   98.589996   
2024-11-21  228.520004  213.960007   34.990002  404.970001  101.809998   
2024-11-22  229.869995  214.589996   37.419998  406.480011  106.080002   
2024-1

In [22]:
df.index = pd.to_datetime(df.index)

In [54]:
#Normalize the features
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df.values)
scaled_df = pd.DataFrame(scaled_data, index=df.index, columns=df.columns)

In [55]:
print(df.head)

<bound method NDFrame.head of Price        Adj Close                                                  \
Ticker            AAPL         ADI          AI        ANET        CROX   
Date                                                                     
2020-12-17  125.849930  134.495972  117.239998   71.342499   65.269997   
2020-12-18  123.855118  135.225555  137.589996   71.930000   64.379997   
2020-12-21  125.390350  133.382828  160.889999   71.815002   65.620003   
2020-12-22  128.959518  134.963684  177.470001   72.705002   64.760002   
2020-12-23  128.059906  133.972137  161.059998   71.622498   64.589996   
...                ...         ...         ...         ...         ...   
2024-11-19  228.279999  210.429993   32.959999  377.700012   97.980003   
2024-11-20  229.000000  211.009995   32.360001  381.709991   98.589996   
2024-11-21  228.520004  213.960007   34.990002  404.970001  101.809998   
2024-11-22  229.869995  214.589996   37.419998  406.480011  106.080002   
2024-11-

# Preprocess

In [27]:
#featues and target
n_future = 5 #this variable will be used when I need to predict 5 days into the future
n_steps = 30 # last 30 days will be used as the input

In [32]:
#prepare the sequences for training
def create_multiticker_sequences(data, n_steps, n_future):
  X,y = [], []
  for i in range(len(data) - n_steps - n_future):
    X.append(data[i: i + n_steps])
    y.append(data[i + n_steps + n_future - 1]) # predict n_future days ahead
  return np.array(X), np.array(y)

In [33]:
#create sequences
X,y = create_multiticker_sequences(scaled_df.values, n_steps, n_future)

In [34]:
#Train test split
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# LSTM

In [35]:
def build_lstm_model(input_shape):
  model = Sequential([
      LSTM(100, return_sequences=True, input_shape=input_shape),
      Dropout(0.2),
      LSTM(100),
      Dropout(0.2),
      Dense(input_shape[1]) #predict all tickers at once
  ])
  model.compile(optimizer = 'adam', loss = 'mse')
  return model

In [36]:
#train the model
input_shape = (n_steps, scaled_df.shape[1])
lstm_model = build_lstm_model(input_shape)
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 6s 74ms/step - loss: 0.6445 - val_loss: 1.1735
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - loss: 0.3101 - val_loss: 0.8989
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.2212 - val_loss: 0.8388
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 0.2074 - val_loss: 0.7995
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - loss: 0.1959 - val_loss: 0.8217
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 98ms/step - loss: 0.1868 - val_loss: 0.8288
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - loss: 0.1794 - val_loss: 0.8412
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - loss: 0.1762 - val_loss: 0.8538
Epoch 9/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - loss: 0.1843 - val_loss: 0.8422
Epoch 10/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 0.1682 - val_loss: 0.8673


In [37]:
lstm_predictions = lstm_model.predict(X_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


In [44]:
#regression analytics
mse = mean_squared_error(y_test, lstm_predictions)
mae = mean_absolute_error(y_test, lstm_predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, lstm_predictions)

In [45]:
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R2: {r2}')

MSE: 0.8673297743089753
MAE: 0.6759202047541273
RMSE: 0.9313054140876532
R2: -2.3429148533651585


In [46]:
#calculate profit or loss on predicted movement
actual_returns = y_test[1:] - y_test[:-1]
predicted_returns = lstm_predictions[1:] - lstm_predictions[:-1]


In [47]:
strategy_returns = np.sign(predicted_returns) * actual_returns
pnl = np.sum(strategy_returns) #profit and loss
print(pnl)

In [49]:
mean_return = np.mean(strategy_returns)
std_return = np.std(strategy_returns)
sharpe_ratio = mean_return / std_return
print(sharpe_ratio)

-0.0016171967215591555


In [51]:
#Hit ratio is the proportion of the model correctly predicting the direction of the stock movement
hits = np.sign(predicted_returns) == np.sign(actual_returns)
hit_ratio = np.sum(hits) / len(hits)
print(f"LSTM Hit Ratio: {hit_ratio:.4f}")

LSTM Hit Ratio: 44.8063


#TCN